In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
path = "../model/Reranker"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForSequenceClassification.from_pretrained(path)

c:\ProgramData\anaconda3\envs\IR\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
inputs = tokenizer('weather in new york', 'it is cold today in new york', return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 4633, 1999, 2047, 2259,  102, 2009, 2003, 3147, 2651, 1999, 2047,
         2259,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [3]:
with torch.no_grad():
    score = model(**inputs).logits
score

tensor([[7.0222]])

In [4]:
tokenizer.decode(inputs['input_ids'][0])

'[CLS] weather in new york [SEP] it is cold today in new york [SEP]'

In [5]:
inputs = tokenizer('weather in new york', 'I am Linda', return_tensors='pt')
with torch.no_grad():
    score = model(**inputs).logits
score

tensor([[-7.3656]])

In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [7]:
from reranker import RerankerForInference
rk = RerankerForInference.from_pretrained(path)

In [8]:
inputs = rk.tokenize('weather in new york', 'it is cold today in new york', return_tensors='pt')
inputs

{'input_ids': tensor([[ 101, 4633, 1999, 2047, 2259,  102, 2009, 2003, 3147, 2651, 1999, 2047,
         2259,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [9]:
with torch.no_grad():
    score = rk(inputs).logits
score

tensor([[7.0222]])